In [28]:
import os
import json
import re
from operator import itemgetter

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import words
from nltk.corpus import wordnet as wn
from nltk.corpus import framenet as fn
from nltk.corpus import stopwords
from nltk.corpus.reader.framenet import PrettyList
from nltk.stem.snowball import SnowballStemmer
from time import time
# ...
# nltk.download('words')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')

In [29]:
directory_summary_main = r"data/BBC News Summary/News Articles/"
sub_dirs = os.listdir(directory_summary_main)

word_count_dict = {}
root_word_count_dict = {}

total_token_count = 0
max_file_count = 1000

i = 0

for sub in sub_dirs:
    files = os.listdir(directory_summary_main + sub)
    for f in files:
        try:
            text = open(directory_summary_main + sub + "/" + f, "r").read()
            tokens = word_tokenize(text.lower())
            total_token_count = total_token_count + len(tokens)
            for tok in tokens:
                if tok.isalpha():
                    if tok not in word_count_dict:
                        word_count_dict[tok] = 0
                    word_count_dict[tok] = word_count_dict[tok] + 1
            i = i + 1
        except:
            continue

        #     if i % 1000==1:
        #         print(i)
        #     if i > max_file_count:
        #         break

word_freq_dict = {}
for w in word_count_dict:
    word_freq_dict[w] = word_count_dict[w]/total_token_count

word_freq_dict_val = {k: v for k, v in sorted(word_freq_dict.items(), key=lambda x: x[1], reverse= False)}  

with open("word_fequency.json", "w") as outfile:
    json.dump(word_freq_dict_val, outfile)
    
# word_freq_dict_val

In [30]:
def get_event_typing(w, thres = 0.0, m = 3, depth = 3):
    ws = wn.synsets(w)
    
    hyper = lambda s: s.hypernyms()
    types1 = {}
    types2 = {}

    for e in ws:
        w_tmp = e.name().split(".")[0]
        if w_tmp==w:
            t = 0
            c = 0 
            i = 0
            tmp_types = ""
            
            tmp_event_types = list(e.closure(hyper))
            
            for tmp_event in tmp_event_types:
                d = wn.path_similarity(e, tmp_event)
                tmp_event_name = tmp_event.name().split(".")[0]
                
                b = m**i
                t = t + (d/b)
                types2[tmp_event_name] = d
                
                if d and (d >= (thres/t)):
                    tmp_types = tmp_types + ":" + tmp_event_name
                    if c >= depth - 1:
                        break
                    c = c + 1
                    
                i = i + 1
                
            if len(tmp_types) > 0:
                types1[tmp_types] = t
                
    types1 = {k: v for k, v in sorted(types1.items(), key=lambda item: item[1], reverse=True)}
    types2 = {k: v for k, v in sorted(types2.items(), key=lambda item: item[1], reverse=True)}
    
    return types1, types2


def get_event_typing_main(w, pos, thres = 0.0, m = 3, depth = 3):
    if not pos:
        pos = nltk.tag.pos_tag(w.split())[0][1]
    
    types1, types2 = get_event_typing(w, thres, m, depth)
    
    if not (types1 or types2):
        if pos.startswith("V"):    
            w_tmp = WordNetLemmatizer().lemmatize(w, "v")
            types1, types2 = get_event_typing(w_tmp, thres, m, depth)
            if (types1 or types2):
                return types1, types2
            
        if pos.startswith("N"):
            w_tmp = WordNetLemmatizer().lemmatize(w, "n")
            types1, types2 = get_event_typing(w_tmp, thres, m, depth)
            if (types1 or types2):
                return types1, types2
            
        w_tmp = WordNetLemmatizer().lemmatize(w)
        types1, types2 = get_event_typing(w_tmp, thres, m, depth)
        
        if (types1 or types2):
            return types1, types2
        
        stemmer = SnowballStemmer("english")
        w_tmp = stemmer.stem(w)
        types1, types2 = get_event_typing(w_tmp, thres, m, depth)
            
    
    return types1, types2


def replacer(match):
    if match.group(1) is not None:
        return '{} '.format(match.group(1))
    else:
        return ' {}'.format(match.group(2))

def preprocess_text(text):
    rx = re.compile(r'^(\W+)|(\W+)$')
    text = " ".join([rx.sub(replacer, word) for word in raw_text.split()])
    text = re.sub("'", " ' ", text)
    return text

In [65]:
stops = set(stopwords.words('english'))
upper_freq_noun = 0.0001
upper_freq_verb = 0.00001



raw_text = "Pfizer and BioNTech have begun a clinical trial for their Omicron-specific Covid-19 vaccine candidate, they announced in a news release on Tuesday. The study will evaluate the vaccine for safety, tolerability and the level of immune response, as both a primary series and a booster dose, in up to 1420 healthy adults ages 18 to 55."
text = preprocess_text(raw_text)

# print(text)
tokens = text.lower().split()

pos_tags = nltk.tag.pos_tag(text.split())
for x in pos_tags:
    token = x[0].lower()
    pos = x[1]
    if (token not in stops) and ((pos.startswith("V")) or (pos=="NN")) and (token in word_freq_dict_val):
        if (pos.startswith("V")) and (word_freq_dict_val[token] > upper_freq_verb):
            continue
        if (pos=="NN") and (word_freq_dict_val[token] > upper_freq_noun):
            continue
        a, b = get_event_typing_main(token, pos=pos, thres=0.0, m=10, depth=3) 
        if a or b:
#             print("_"*100)
#             print(token, end=", ")
            print(token, pos)
#         if a:
#             print(a)
#             print("\n")
#         if b:
#             print(b)
        


vaccine NN
candidate NN
study NN
evaluate VB
vaccine NN
safety NN
dose NN


In [56]:
word_freq_dict_val["ash"]

6.163847391357053e-06